In [ ]:
from model import *
from dataloader import *
import glob
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import gc

In [2]:
# fix for tensorflow pipeline
def _parse_function(filepath, label): 
  image_string = tf.io.read_file(filepath)
  image_decoded = tf.image.decode_jpeg(image_string, channels=3)
  image_resized = tf.image.resize(image_decoded, [500, 500])
  image_normalized = tf.cast(image_resized, tf.float32) / 255.0

  # Convert image_normalized to a NumPy array for calories function
  def convert_and_call_calories(image_tensor, label_tensor):
      image_array = image_tensor.astype(np.uint8)
      predictions = np.array(calories(label_tensor, image_array)[:2], dtype=np.float32)
      return predictions

  predictions = tf.numpy_function(convert_and_call_calories,
                                  [image_normalized, label],
                                  tf.float32)  # Assuming calories returns a float32 tensor
  out = (image_normalized, predictions)
  return out


In [3]:
data = {
  "filepath":[],
  "class":[]
}
for i,fr in enumerate(fruits): # fruits variable in dataloader.py
  fruit_img = glob.glob(f"FooDD\{fr}\**\*.jpg")
  fruit_label = [i+1]*len(fruit_img)
  data["filepath"].extend(fruit_img)
  data["class"].extend(fruit_label)

visulization_df = pd.DataFrame(data)
print(visulization_df)

                                               filepath  class
0     FooDD\Apple\1-Samsung-S4-Light Environment\1 (...      1
1     FooDD\Apple\1-Samsung-S4-Light Environment\1 (...      1
2     FooDD\Apple\1-Samsung-S4-Light Environment\1 (...      1
3     FooDD\Apple\1-Samsung-S4-Light Environment\1 (...      1
4     FooDD\Apple\1-Samsung-S4-Light Environment\1 (...      1
...                                                 ...    ...
1602  FooDD\Tomato\6-CanonSD-1400-Dark Environment\I...      7
1603  FooDD\Tomato\6-CanonSD-1400-Dark Environment\I...      7
1604  FooDD\Tomato\6-CanonSD-1400-Dark Environment\I...      7
1605  FooDD\Tomato\6-CanonSD-1400-Dark Environment\I...      7
1606  FooDD\Tomato\6-CanonSD-1400-Dark Environment\I...      7

[1607 rows x 2 columns]


In [4]:
valid_img = 100
BATCH_SIZE = 8
EPOCHS = 10
NAME = "Model_arch_VIT"

In [5]:
dataset_tf = tf.data.Dataset.from_tensor_slices((visulization_df['filepath'], visulization_df['class']))
dataset_tf = dataset_tf.map(_parse_function)
dataset_tf = dataset_tf.map(lambda img, lbl: (tf.ensure_shape(img, [500, 500, 3]), tf.ensure_shape(lbl, [2,])) ).shuffle(len(dataset_tf))
valid_tf = dataset_tf.take(valid_img)
train_tf = dataset_tf.skip(valid_img)

In [6]:
# converting tf.Dataset to numpy arrays since calories function slows down the tf function.
# untill the calories function is fully rewritten using tf and keras, it is not recommened to use them
train_dataset = train_tf.as_numpy_iterator()
valid_dataset = valid_tf.as_numpy_iterator()

In [7]:
X_train = []
Y_train = []
X_valid = []
Y_valid = []

for x,y in train_dataset:
    X_train.append(x)
    Y_train.append(y)

for x,y in valid_dataset:
    X_valid.append(x)
    Y_valid.append(y)

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

In [8]:
X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape

((1507, 500, 500, 3), (1507, 2), (100, 500, 500, 3), (100, 2))

In [9]:
img_h,img_w,img_c = (500,500,3)

In [ ]:
model = create_fruit_calorie_model((img_h,img_w,img_c),2)

In [ ]:
history = model.fit(X_train, Y_train, epochs=EPOCHS, batch_size = BATCH_SIZE, validation_data=(X_valid, Y_valid))

In [ ]:
import matplotlib.pyplot as plt

# Hardcoded values from the extracted data
losses = history.history['loss']
maes = history.history['mae']
val_losses = history.history['val_loss']
val_maes = history.history['val_mae']
epochs = range(len(losses))

# Plotting
plt.figure(figsize=(14, 6))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(epochs, losses, label='Loss', marker='o')
plt.plot(epochs, val_losses, label='Val Loss', marker='o')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# MAE Plot
plt.subplot(1, 2, 2)
plt.plot(epochs, maes, label='MAE', marker='o')
plt.plot(epochs, val_maes, label='Val MAE', marker='o')
plt.title('MAE vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error (MAE)')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

# Hyperparameter Tuning

In [ ]:
import keras_tuner as kt
from keras_tuner import HyperModel
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import pandas as pd

In [ ]:
# Flexible Model Builder with model_length and Dropout
class MyHyperModel(HyperModel):
    def __init__(self, input_shape, out_shape, **kwargs):
        super().__init__(**kwargs)
        self.input_shape = input_shape
        self.out_shape = out_shape

    def build(self, hp):
        tf.keras.backend.clear_session()
        gc.collect()
        # Input layer
        inp = layers.Input(shape=self.input_shape)

        # Choose block configurations
        model_length = hp.Int('model_length', min_value=1, max_value=5)  # Number of blocks to stack
        dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)  # Dropout rate

        # Initial feature extraction block configuration
        f = hp.Int('filters', min_value=8, max_value=64, step=8)
        x = conv_feature_block(inp, f=f)

        # Loop through blocks based on model_length
        for i in range(model_length):
            # Block-specific hyperparameters
            use_pooling_reduction = hp.Boolean(f'use_pooling_reduction_{i}')
            use_conv_reduction = hp.Boolean(f'use_conv_reduction_{i}')
            use_localization = hp.Boolean(f'use_localization_{i}')

            # Apply pooling reduction block if selected
            if use_pooling_reduction:
                x = pooling_reduction_block(x, f=f, n=hp.Int(f'pooling_reduction_n_{i}', min_value=1, max_value=3))
            
            # Apply convolutional reduction block if selected
            if use_conv_reduction:
                x = conv_reduction_block(x, f=f, n=hp.Int(f'conv_reduction_n_{i}', min_value=2, max_value=4))

            # Apply localization block if selected
            if use_localization:
                x = localization_block(x, f=f)

            # Add dropout after each block if specified
            if dropout_rate > 0:
                x = layers.Dropout(dropout_rate)(x)

        # Final layers (Dense or other classifiers)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(64, activation='relu')(x)
        out = layers.Dense(self.out_shape, activation='relu')(x)

        model = Model(inputs=inp, outputs=out)
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        
        return model

In [ ]:
hypermodel = MyHyperModel(input_shape = (img_h,img_w,img_c), out_shape = 2)

tuner = kt.Hyperband(
    hypermodel,
    objective='val_mae',
    max_epochs=10,
    factor=2,
    directory='vit_tuning',
    project_name='regression_vit',
    overwrite=True
)

In [13]:
tuner.search(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_valid, Y_valid))

Trial 5 Complete [03h 59m 20s]
val_mae: 25.27922026316325

Best val_mae So Far: 10.087055842081705
Total elapsed time: 06h 58m 15s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
4                 |2                 |model_length
0.2               |0.2               |dropout_rate
32                |8                 |filters
False             |False             |use_pooling_reduction_0
True              |False             |use_conv_reduction_0
False             |True              |use_localization_0
2                 |2                 |pooling_reduction_n_0
False             |False             |use_pooling_reduction_1
False             |False             |use_conv_reduction_1
True              |False             |use_localization_1
3                 |None              |conv_reduction_n_0
4                 |None              |conv_reduction_n_1

Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 937s 5s/step - loss: 897.6866 - mae: 20.7567 - val_loss: 373.5116 - va

KeyboardInterrupt: 

In [18]:
del X_train
del Y_train
del X_valid
del Y_valid

In [19]:
import gc
gc.collect()

9707

NameError: name 'tuner' is not defined

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_mae = best_model.evaluate(valid_tf)
print(f"Test MAE: {test_mae}")